# PURPOSE OF THIS NOTEBOOK
This is the base code to write the functions to extract the raw text from resumes and append to our dataframe. 

TODO: Take the functions into a .py folder and use it as a script 

In [16]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

## Model goal

My goal is to load in a model from Hugging Face and train with the ONET data, This will allow me to utilize the power of the models on hugging face. I would need to start training this model on my data to see how it would preform.

In [17]:
# Import the ONET data. 
onet_df = pd.read_csv("../Data/Updated_ONET_Alt_Titles.csv")

In [18]:
onet_df

,O*NET-SOC Code,Title,Reported Job Title,Shown in My Next Move
0,11-1011.00,Chief Executives,Chief Diversity Officer (CDO),N
1,11-1011.00,Chief Executives,Chief Executive Officer (CEO),Y
2,11-1011.00,Chief Executives,Chief Financial Officer (CFO),Y
3,11-1011.00,Chief Executives,Chief Nursing Officer,N
4,11-1011.00,Chief Executives,Chief Operating Officer (COO),Y
...,...,...,...,...
44540,55-3019.00,Military Enlisted Tactical Operations and Air/...,Tactical Debriefer,NaN
44541,55-3019.00,Military Enlisted Tactical Operations and Air/...,Tactical/Mobile (Tacmobile) Ashore Analysis Sy...,NaN
44542,55-3019.00,Military Enlisted Tactical Operations and Air/...,Target Aircraft Technician,NaN
44543,55-3019.00,Military Enlisted Tactical Operations and Air/...,Technical Surveillance Countermeasures (TSCM) ...,NaN


## The goal of the tokenizer

The tokenizer will be tokenizing the job titles and the reported job titles, I don't know if I should do these seperately or together. In theory, I should have these pairings be tokenized together. **I SHOULD READ INTO THE TOKENIZER TO UNDERSTAND HOW I SHOULD APPROACH THIS** 

In [19]:
# This is text extraction, Code will be deprecated for now until I find use for it later. 
text_data = []
file_count=0

for sample in tqdm(onet_df['Title']):
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'../data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 44545/44545 [00:00<00:00, 981860.41it/s]


In [20]:
# Start with the tokenizer 
paths = [str(x) for x in Path('../data/').glob('**/*.txt')]

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2, special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [2]:
# Code to load in the pretrained model from hugging face. 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained('model_name')
tokenizer = AutoTokenizer.from_pretrained('model_name')

features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

/opt/anaconda3/envs/Resume/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: model_name is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

## The validation data. 

Since this dataset that I'm using to test does not have ONET codes attached to them, I would have to source another dataset that does. Thankfully ONET does have such data that I can use for training. For each ONET code there's common job names appended to them, I could use that data with the pretained model. That being said this creates some challenges. 
1. For obsure and odd job names that are not listed in the common job names will almost guarentee bad performance. Given that the goal is to prove that we can get generally good performance with publicly available data, this is something I'm willing to sacrifice if it means that it will work correctly at least 80% of the time. 

I have sourced a table with common names linked with each onet code. I can use this data on the pretrained model to see how it classifies each of the common names linked with ONET codes. 

In [6]:
# Start with removing all the duplicates 
onet_df

,O*NET-SOC Code,Title,Reported Job Title,Shown in My Next Move
0,11-1011.00,Chief Executives,Chief Diversity Officer (CDO),N
1,11-1011.00,Chief Executives,Chief Executive Officer (CEO),Y
2,11-1011.00,Chief Executives,Chief Financial Officer (CFO),Y
3,11-1011.00,Chief Executives,Chief Nursing Officer,N
4,11-1011.00,Chief Executives,Chief Operating Officer (COO),Y
...,...,...,...,...
44540,55-3019.00,Military Enlisted Tactical Operations and Air/...,Tactical Debriefer,NaN
44541,55-3019.00,Military Enlisted Tactical Operations and Air/...,Tactical/Mobile (Tacmobile) Ashore Analysis Sy...,NaN
44542,55-3019.00,Military Enlisted Tactical Operations and Air/...,Target Aircraft Technician,NaN
44543,55-3019.00,Military Enlisted Tactical Operations and Air/...,Technical Surveillance Countermeasures (TSCM) ...,NaN


## Next steps

After loading in the pretrained model, I want to load the data into this model and test the performance while holding back around 30% of the data for testing. After I complete the testing phase of the model performance, I will start the next step and pull, or train a model or a script to pull the job titles from a resume and apply into the model to classify into an appropriote ONET code. 